In [324]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.preprocessing import OneHotEncoder

In [325]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import linear_model

In [326]:
df = pd.read_csv('Interview.csv')

In [327]:
df.columns

Index([u'Date of Interview', u'Client name', u'Industry', u'Location',
       u'Position to be closed', u'Nature of Skillset', u'Interview Type',
       u'Name(Cand ID)', u'Gender', u'Candidate Current Location',
       u'Candidate Job Location', u'Interview Venue',
       u'Candidate Native location',
       u'Have you obtained the necessary permission to start at the required time',
       u'Hope there will be no unscheduled meetings',
       u'Can I Call you three hours before the interview and follow up on your attendance for the interview',
       u'Can I have an alternative number/ desk number. I assure you that I will not trouble you too much',
       u'Have you taken a printout of your updated resume. Have you read the JD and understood the same',
       u'Are you clear with the venue details and the landmark.',
       u'Has the call letter been shared', u'Expected Attendance',
       u'Observed Attendance', u'Marital Status', u'Unnamed: 23',
       u'Unnamed: 24', u'Unnamed: 2

In [328]:
df.drop(['Date of Interview','Name(Cand ID)','Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26','Unnamed: 27'], axis=1, inplace=True)

In [329]:
columns_to_lower = ['Client name','Industry','Location','Position to be closed','Nature of Skillset','Interview Type','Gender','Candidate Current Location']
for column in columns_to_lower:
    df[column] = df[column].astype(str).str.lower()

In [330]:
df.describe()

,Client name,Industry,Location,Position to be closed,Nature of Skillset,Interview Type,Gender,Candidate Current Location,Candidate Job Location,Interview Venue,...,Have you obtained the necessary permission to start at the required time,Hope there will be no unscheduled meetings,Can I Call you three hours before the interview and follow up on your attendance for the interview,Can I have an alternative number/ desk number. I assure you that I will not trouble you too much,Have you taken a printout of your updated resume. Have you read the JD and understood the same,Are you clear with the venue details and the landmark.,Has the call letter been shared,Expected Attendance,Observed Attendance,Marital Status
count,1234,1234,1234,1234,1234,1234,1234,1234,1233,1233,...,1029,986,986,986,985,985,988,1228,1233,1233
unique,16,8,10,8,83,7,3,9,7,7,...,7,7,5,6,8,7,12,7,8,2
top,standard chartered bank,bfsi,chennai,routine,java/j2ee/struts/hibernate,scheduled walk in,male,chennai,Chennai,Chennai,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Single
freq,904,949,841,1023,220,456,965,841,893,852,...,917,949,951,936,940,946,932,882,701,767


In [331]:
skills = list(set(df["Nature of Skillset"]))
skills = [re.split(", |,|/ |/", i) for i in skills]
skills = [skill for line in skills for skill in line]
skills = sorted(list(set(skills)))
skills

['- sapbo',
 '10.00 am',
 '11.30 am',
 '12.30 pm',
 '9.00 am',
 '9.30 am',
 'accounting operations',
 'als testing',
 'aml',
 'analytical r & d',
 'analytical r&d',
 'automation testing java',
 'banking operations',
 'basesas program',
 'biosimilars',
 'biosimiliars',
 'biosimillar',
 'cdd',
 'cdd kyc',
 'core java',
 'cots',
 'cots developer',
 'dot net',
 'emea',
 'etl',
 'fresher',
 'generic drugs \xe2\x80\x93 ra',
 'global labelling',
 'hadoop',
 'hibernate',
 'informatica',
 'j2ee',
 'java',
 'java ',
 'java developer',
 'java j2ee',
 'java jsf',
 'java tech lead',
 'java-sas',
 'jsf',
 'kyc',
 'l & l',
 'lcm -manager',
 'lending & liability',
 'lending and liabilities',
 'lending&liablities',
 'licensing \xe2\x80\x93 ra',
 'manager',
 'ms exchange)',
 'nan',
 'oracle',
 'oracle plsql',
 'product control',
 'production',
 'production support - sccm',
 'publishing',
 'ra label',
 'ra publishing',
 'regulatory',
 'reporting',
 'routine',
 'sas',
 'sccm',
 'sccm \xe2\x80\x93 sharepoi

In [332]:
invalid_skills = ['10.00 am','11.30 am','12.30 pm','9.00 am','9.30 am']
df = df.loc[[ skill not in invalid_skills for skill in df["Nature of Skillset"]]]

In [333]:
df["Nature of Skillset"].value_counts().sort_index()

- sapbo, informatica                       4
accounting operations                     86
als testing                               15
aml/kyc/cdd                               84
analytical r & d                          13
analytical r&d                             3
automation testing java                    7
banking operations                        24
basesas program/ reporting                 1
biosimilars                                1
biosimiliars                               6
biosimillar                                3
cdd kyc                                   52
core java                                 17
cots                                       4
cots developer                            13
dot net                                    9
emea                                       6
etl                                        9
fresher                                   86
generic drugs – ra                         4
global labelling                           6
hadoop    

In [334]:
col_names =  ['lookup_regex', 'extract_regex', 'column_to_lookup','column_to_complete','value_type','value_offset','extra_condition']
diccionario  = pd.DataFrame(columns = col_names)
diccionario.loc[len(diccionario)] = ['lending & liability|lending and liabilities|lending&liablities', '', 'Nature of Skillset', 'Nature of Skillset', 'string','l & l', {'data_column' : 'Nature of Skillset', 'value' : ['lending & liability','lending and liabilities','lending&liablities'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['tech lead- mednet|tech lead-mednet', '', 'Nature of Skillset', 'Nature of Skillset', 'string','tech lead/mednet', {'data_column' : 'Nature of Skillset', 'value' : ['tech lead- mednet','tech lead-mednet'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['biosimilars|biosimiliars', '', 'Nature of Skillset', 'Nature of Skillset', 'string','biosimillar', {'data_column' : 'Nature of Skillset', 'value' : ['biosimilars','biosimiliars'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java j2ee|j2ee|java ,j2ee', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/j2ee', {'data_column' : 'Nature of Skillset', 'value' : ['java j2ee','j2ee','java ,j2ee'], 'condition_func': (lambda x,y: x.isin(y))}]
#diccionario.loc[len(diccionario)] = ['sccm- networking|sccm-\(network', '', 'Nature of Skillset', 'Nature of Skillset', 'string','sccm/networking', {'data_column' : 'Nature of Skillset', 'value' : ['sccm- networking', 'sccm-(network'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java developer', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/developer', {'data_column' : 'Nature of Skillset', 'value' : ['java developer'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java jsf', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/jsf', {'data_column' : 'Nature of Skillset', 'value' : ['java jsf'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java tech lead', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/tech lead', {'data_column' : 'Nature of Skillset', 'value' : ['java tech lead'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['core java|java ', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java', {'data_column' : 'Nature of Skillset', 'value' : ['core java','java '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['analytical r & d', '', 'Nature of Skillset', 'Nature of Skillset', 'string','analytical r&d', {'data_column' : 'Nature of Skillset', 'value' : ['analytical r & d'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['automation testing java', '', 'Nature of Skillset', 'Nature of Skillset', 'string','automation testing/java', {'data_column' : 'Nature of Skillset', 'value' : ['automation testing java'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['cots developer', '', 'Nature of Skillset', 'Nature of Skillset', 'string','cots/developer', {'data_column' : 'Nature of Skillset', 'value' : ['cots developer'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['ra label', '', 'Nature of Skillset', 'Nature of Skillset', 'string','ra/label', {'data_column' : 'Nature of Skillset', 'value' : ['ra label'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['ra publishing', '', 'Nature of Skillset', 'Nature of Skillset', 'string','ra/publishing', {'data_column' : 'Nature of Skillset', 'value' : ['ra publishing'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['sccm-\(network, sharepoint,ms exchange\)', '', 'Nature of Skillset', 'Nature of Skillset', 'string','sccm-network, sharepoint,ms exchange', {'data_column' : 'Nature of Skillset', 'value' : ['sccm-(network, sharepoint,ms exchange)'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['tl|technical lead', '', 'Nature of Skillset', 'Nature of Skillset', 'string','tech lead', {'data_column' : 'Nature of Skillset', 'value' : ['tl','technical lead'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['t-24 developer', '', 'Nature of Skillset', 'Nature of Skillset', 'string','t 24/developer', {'data_column' : 'Nature of Skillset', 'value' : ['t-24 developer'], 'condition_func': (lambda x,y: x.isin(y))}]

In [335]:
def parse(string,pattern):
    matches = re.search(pattern["lookup_regex"], string)
    parse_data = None
    if matches != None:
        if (pattern["value_type"] == 'float') and (pattern["extract_regex"] != ''):
            parse_data = float( re.search(pattern["extract_regex"], matches.group(0)).group(0).replace(".","") ) + float(pattern["value_offset"])
        elif (pattern["value_type"] == 'int') and (pattern["extract_regex"] != ''):
            parse_data = int( re.search(pattern["extract_regex"], matches.group(0)).group(0).replace(".","") ) + int(pattern["value_offset"])
        elif (pattern["value_type"] == 'string') and (pattern["extract_regex"] != ''):
            parse_data = re.search(pattern["extract_regex"], matches.group(0)).group(0) + pattern["value_offset"]
        elif pattern["value_type"] == 'float':
            parse_data = float(pattern["value_offset"])
        elif pattern["value_type"] == 'int':
            parse_data = int(pattern["value_offset"])
        else:
            parse_data = pattern["value_offset"]
    else:
        parse_data = np.nan;
    return parse_data

def df_parser(data,parse_dictionary):
    for pattern_row_index, pattern in parse_dictionary.iterrows():
        #rows_to_parse = (data[pattern["column_to_complete"]].isnull() == True) & (pattern['extra_condition']['condition_func'] (data[pattern['extra_condition']['data_column']],pattern['extra_condition']['value']) == True) 
        rows_to_parse = pattern['extra_condition']['condition_func'] (data[pattern['extra_condition']['data_column']],pattern['extra_condition']['value']) == True 
        info_to_parse = data[rows_to_parse][pattern["column_to_lookup"]]
        valid_info = info_to_parse[info_to_parse.apply(lambda a : (type(a) is str) and (a != ''))]
        data.loc[rows_to_parse, pattern["column_to_complete"]] = valid_info.apply(parse, args=([pattern]))

In [336]:
df_parser(df,diccionario)

In [337]:
df['Nature of Skillset'].value_counts().sort_index()

- sapbo, informatica                     4
accounting operations                   86
als testing                             15
aml/kyc/cdd                             84
analytical r&d                          16
automation testing/java                  7
banking operations                      24
basesas program/ reporting               1
biosimillar                             10
cdd kyc                                 52
cots                                     4
cots/developer                          13
dot net                                  9
emea                                     6
etl                                      9
fresher                                 86
generic drugs – ra                       4
global labelling                         6
hadoop                                  12
java                                    48
java, j2ee                               2
java, spring, hibernate                  2
java, sql                                3
java, xml, 

In [338]:
#df['Nature of Skillset'] = [re.split(" , | ,|, |,| /|/ |/| - | -|- |-| – ", i) for i in df['Nature of Skillset']]
#df.loc[df['Nature of Skillset'].astype(str).str.contains('/')]['Nature of Skillset']

In [339]:
df['Nature of Skillset']

0                  routine
1                  routine
2                  routine
3                  routine
4                  routine
5                  routine
6                  routine
7                  routine
8                  routine
9                  routine
10                 routine
11                 routine
12                 routine
13                 routine
14                 routine
15                 routine
16                 routine
17                 routine
18                 routine
19                 routine
20                 routine
21                 routine
22                 routine
23                 routine
24                 routine
25                 routine
26                 routine
27                 routine
28                 routine
29                 routine
               ...        
1204                  java
1205                  java
1206                  java
1207                  java
1208                  java
1209                  java
1

In [340]:
df['n_skills'] = df['Nature of Skillset'].apply(len)

In [341]:
df["Industry"].value_counts()

bfsi                        942
pharmaceuticals             165
it products and services     45
it services                  23
electronics                  23
telecom                      17
it                           11
nan                           1
Name: Industry, dtype: int64

In [342]:
df["Client name"].value_counts()

standard chartered bank            897
hospira                             75
pfizer                              75
aon hewitt                          28
flextronics                         23
anz                                 22
hewitt                              20
ust                                 18
standard chartered bank chennai     17
prodapt                             17
astrazeneca                         15
williams lea                        11
barclays                             5
aon hewitt gurgaon                   2
woori bank                           1
﻿﻿                                   1
Name: Client name, dtype: int64

In [343]:
df["Position to be closed"].value_counts()

routine                1016
niche                   163
dot net                  18
trade finance            11
aml                       8
production- sterile       5
selenium testing          5
nan                       1
Name: Position to be closed, dtype: int64

In [344]:
relevant_cols = ["Position to be closed"]
df = df.dropna(subset=relevant_cols)

In [345]:
df["Position to be closed"].value_counts()

routine                1016
niche                   163
dot net                  18
trade finance            11
aml                       8
production- sterile       5
selenium testing          5
nan                       1
Name: Position to be closed, dtype: int64

In [346]:
df["Interview Type"].value_counts()

scheduled walk in    449
scheduled            371
walkin               189
scheduled walkin     189
walkin                27
nan                    1
sceduled walkin        1
Name: Interview Type, dtype: int64

In [347]:
df.loc[df["Interview Type"].astype(str).str.contains('sceduled'),"Interview Type"]= "scheduled walkin"

### One hot encoding

In [348]:
df["Nature of Skillset"].value_counts()

java/j2ee/struts/hibernate              220
accounting operations                    86
fresher                                  86
aml/kyc/cdd                              84
java/j2ee                                63
cdd kyc                                  52
java                                     48
routine                                  47
oracle                                   43
java/spring/hibernate/jsf                42
l & l                                    33
sas                                      27
java/developer                           25
oracle plsql                             25
banking operations                       24
analytical r&d                           16
senior software engineer-mednet          15
t 24/developer                           15
sccm                                     15
als testing                              15
cots/developer                           13
sr automation testing                    13
hadoop                          

In [349]:
onehot_skills = df["Nature of Skillset"].str.split(" , | ,|, |,| /|/ |/| - | -|- |-| – ", expand=True).stack().str.get_dummies().sum(level=0)
onehot_skills

,accounting operations,als testing,aml,analytical r&d,automation testing,banking operations,basesas program,biosimillar,cdd,cdd kyc,...,sharepoint,spring,sql,sr automation testing,struts,submission management,t 24,tech lead,testing,xml
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [350]:
onehot_skills.describe()

,accounting operations,als testing,aml,analytical r&d,automation testing,banking operations,basesas program,biosimillar,cdd,cdd kyc,...,sharepoint,spring,sql,sr automation testing,struts,submission management,t 24,tech lead,testing,xml
count,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,...,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000,1227.000000
mean,0.070090,0.012225,0.068460,0.013040,0.005705,0.019560,0.000815,0.008150,0.068460,0.042380,...,0.001630,0.038305,0.009780,0.010595,0.181744,0.001630,0.012225,0.015485,0.008965,0.002445
std,0.255402,0.109933,0.252636,0.113492,0.075346,0.138539,0.028548,0.089945,0.252636,0.201536,...,0.040357,0.192009,0.098449,0.102427,0.385791,0.040357,0.109933,0.123522,0.094297,0.049406
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
